In [1]:
import spotipy
import spotifyfunctions
import pandas as pd
import numpy as np
from spotipy.oauth2 import SpotifyClientCredentials #To access authorised Spotify data

In [2]:
lyrics = pd.read_csv("FINAL_v1.csv")

,Unnamed: 0,album_title,artist,track_title,lyrics
0,1,Girl Code,City Girls,What We Doin',"(The 90's), Yup, haha, House super clean (Clea..."
1,2,Girl Code,City Girls,Season (Ft. Lil Baby),"It's that season (Y'all know what time it is),..."
2,3,Girl Code,City Girls,Broke Boy,"If you a broke boy, stay the fuck out my way, ..."
3,4,Girl Code,City Girls,Clout Chasin',"Yung Miami, ho, JT on the track, Told that lil..."
4,5,Girl Code,City Girls,Intro (#FREEJT),"You have pre-paid call, You will not be charge..."


In [3]:
client_id = "3030c6beb3ea48d393146c3a4c559684"
client_secret = "f0e01dbe13784094b3bb532617f14765"
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager) #spotify object to access API

In [24]:
def albumSongs(uri, spotify_albums, album_names, album_count):
    album = uri #assign album uri to a_name
    spotify_albums[album] = {} #Creates dictionary for that specific album
#Create keys-values of empty lists inside nested dictionary for album
    spotify_albums[album]['album'] = [] #create empty list
    spotify_albums[album]['track_number'] = []
    spotify_albums[album]['id'] = []
    spotify_albums[album]['name'] = []
    spotify_albums[album]['uri'] = []
    tracks = sp.album_tracks(album) #pull data on album tracks
    for n in range(len(tracks['items'])): #for each song track
        spotify_albums[album]['album'].append(album_names[album_count]) #append album name tracked via album_count
        spotify_albums[album]['track_number'].append(tracks['items'][n]['track_number'])
        spotify_albums[album]['id'].append(tracks['items'][n]['id'])
        spotify_albums[album]['name'].append(tracks['items'][n]['name'])
        spotify_albums[album]['uri'].append(tracks['items'][n]['uri'])

In [25]:
def audio_features(album, spotify_albums):
    #Add new key-values to store audio features
    spotify_albums[album]['acousticness'] = []
    spotify_albums[album]['danceability'] = []
    spotify_albums[album]['energy'] = []
    spotify_albums[album]['instrumentalness'] = []
    spotify_albums[album]['liveness'] = []
    spotify_albums[album]['loudness'] = []
    spotify_albums[album]['speechiness'] = []
    spotify_albums[album]['tempo'] = []
    spotify_albums[album]['valence'] = []
    spotify_albums[album]['popularity'] = []
    #create a track counter
    track_count = 0
    for track in spotify_albums[album]['uri']:
        #pull audio features per track
        features = sp.audio_features(track)
        
        #Append to relevant key-value
        spotify_albums[album]['acousticness'].append(features[0]['acousticness'])
        spotify_albums[album]['danceability'].append(features[0]['danceability'])
        spotify_albums[album]['energy'].append(features[0]['energy'])
        spotify_albums[album]['instrumentalness'].append(features[0]['instrumentalness'])
        spotify_albums[album]['liveness'].append(features[0]['liveness'])
        spotify_albums[album]['loudness'].append(features[0]['loudness'])
        spotify_albums[album]['speechiness'].append(features[0]['speechiness'])
        spotify_albums[album]['tempo'].append(features[0]['tempo'])
        spotify_albums[album]['valence'].append(features[0]['valence'])
        #popularity is stored elsewhere
        pop = sp.track(track)
        spotify_albums[album]['popularity'].append(pop['popularity'])
        track_count+=1

In [28]:
def get_album_data (artist_name):
    
    result = sp.search(artist_name) #search query
    result_length = len(result['tracks']['items'])

    for i in range(result_length):
        #print (result['tracks']['items'][i]['artists'][0])
        artist_name_on_spotify = (result['tracks']['items'][i]['artists'][0]['name'])
    
        if artist_name_on_spotify == artist_name:
            artist_uri = result['tracks']['items'][i]['artists'][0]['uri']
            break
            
    #Pull all of the artist's albums
    sp_albums = sp.artist_albums(artist_uri, album_type='album')
    #Store artist's albums' names' and uris in separate lists
    album_names = []
    album_uris = []
    for i in range(len(sp_albums['items'])):
        album_names.append(sp_albums['items'][i]['name'])
        album_uris.append(sp_albums['items'][i]['uri'])
    
    spotify_albums = {}
    album_count = 0
    for i in album_uris: #each album
        albumSongs(i, spotify_albums, album_names, album_count)
        #print("Album " + str(album_names[album_count]) + " songs has been added to spotify_albums dictionary")
        album_count+=1 #Updates album count once all tracks have been added
        
    request_count = 0 
        
    for i in spotify_albums:
        audio_features(i, spotify_albums)
        request_count+=1
        #if request_count % 5 == 0:
            #print(str(request_count) + " playlists completed")
            #time.sleep(np.random.uniform(sleep_min, sleep_max))
            #print('Loop #: {}'.format(request_count))
            #print('Elapsed Time: {} seconds'.format(time.time() - start_time))
            
    dic_df = {}
    dic_df['album'] = []
    dic_df['track_number'] = []
    dic_df['id'] = []
    dic_df['name'] = []
    dic_df['uri'] = []
    dic_df['acousticness'] = []
    dic_df['danceability'] = []
    dic_df['energy'] = []
    dic_df['instrumentalness'] = []
    dic_df['liveness'] = []
    dic_df['loudness'] = []
    dic_df['speechiness'] = []
    dic_df['tempo'] = []
    dic_df['valence'] = []
    dic_df['popularity'] = []
    for album in spotify_albums: 
        for feature in spotify_albums[album]:
            dic_df[feature].extend(spotify_albums[album][feature])
            
    df = pd.DataFrame.from_dict(dic_df)
    
    df = df.sort_values('popularity', ascending=False).drop_duplicates('name').sort_index()
    
    return (df)

In [36]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

In [37]:
eminem = get_album_data("Eminem")
eminem

,album,track_number,id,name,uri,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,popularity
0,Music To Be Murdered By,1,7ccTcabbJlCJiIqtrSSwBk,Premonition - Intro,spotify:track:7ccTcabbJlCJiIqtrSSwBk,0.02580,0.847,0.637,0.000648,0.3820,-5.193,0.3670,165.979,0.5110,71
1,Music To Be Murdered By,2,0q2vG0UVuy6ajjcpkQHdZM,Unaccommodating (feat. Young M.A),spotify:track:0q2vG0UVuy6ajjcpkQHdZM,0.01690,0.852,0.752,0.000000,0.3940,-5.793,0.0597,89.950,0.3890,73
2,Music To Be Murdered By,3,0GQ5bFTVFFKpwNPc7KwQnB,"You Gon’ Learn (feat. Royce Da 5'9"" & White Gold)",spotify:track:0GQ5bFTVFFKpwNPc7KwQnB,0.06140,0.680,0.820,0.000000,0.5970,-4.197,0.1510,140.033,0.2430,74
3,Music To Be Murdered By,4,4EmunTy7kNBYQivOa8F6b8,Alfred - Interlude,spotify:track:4EmunTy7kNBYQivOa8F6b8,0.87800,0.429,0.231,0.000000,0.2790,-20.430,0.4020,74.545,0.9140,67
4,Music To Be Murdered By,5,0VSzREd1OjEWJ9tXoFHRQH,Those Kinda Nights (feat. Ed Sheeran),spotify:track:0VSzREd1OjEWJ9tXoFHRQH,0.02800,0.794,0.637,0.000000,0.0919,-6.290,0.3390,103.868,0.3930,76
5,Music To Be Murdered By,6,7Dj2wqPYXfSRFwa6qyOztr,In Too Deep,spotify:track:7Dj2wqPYXfSRFwa6qyOztr,0.03870,0.720,0.767,0.000000,0.2020,-5.839,0.2140,88.966,0.5950,71
6,Music To Be Murdered By,7,7FIWs0pqAYbP91WWM0vlTQ,Godzilla (feat. Juice WRLD),spotify:track:7FIWs0pqAYbP91WWM0vlTQ,0.14500,0.808,0.745,0.000000,0.2920,-5.260,0.3420,165.995,0.8290,91
7,Music To Be Murdered By,8,5SiZJoLXp3WOl3J4C8IK0d,Darkness,spotify:track:5SiZJoLXp3WOl3J4C8IK0d,0.00998,0.671,0.623,0.000008,0.6430,-7.161,0.3080,75.055,0.1950,74
8,Music To Be Murdered By,9,5CpsIb7SMFHjmAG8RvIWfq,Leaving Heaven (feat. Skylar Grey),spotify:track:5CpsIb7SMFHjmAG8RvIWfq,0.17500,0.661,0.748,0.000000,0.2540,-4.809,0.2060,171.983,0.5090,72
9,Music To Be Murdered By,10,5mZXWEH2eh8zMZGCxT5aW0,"Yah Yah (feat. Royce Da 5'9"", Black Thought, Q...",spotify:track:5mZXWEH2eh8zMZGCxT5aW0,0.14000,0.458,0.966,0.000000,0.1100,-2.107,0.3840,183.586,0.3560,69


,album,track_number,id,name,uri,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,popularity
0,4:44,1,2kh78Yka2tkYOx3XF9DVpu,Kill Jay Z,spotify:track:2kh78Yka2tkYOx3XF9DVpu,0.133000,0.542,0.821,0.000001,0.5700,-6.458,0.3200,84.425,0.2860,54
1,4:44,2,6JpN5w95em8SODPiM7W2PH,The Story of O.J.,spotify:track:6JpN5w95em8SODPiM7W2PH,0.283000,0.741,0.718,0.000002,0.2300,-5.823,0.4150,165.848,0.5760,65
2,4:44,3,2tqoJQbv6dNNOi0PCCjuse,Smile,spotify:track:2tqoJQbv6dNNOi0PCCjuse,0.303000,0.779,0.570,0.000000,0.2180,-8.752,0.4690,99.878,0.4670,55
3,4:44,4,71oM9w2BbJxzTpxzWIE0lw,Caught Their Eyes,spotify:track:71oM9w2BbJxzTpxzWIE0lw,0.166000,0.813,0.837,0.000000,0.0835,-6.356,0.2980,84.965,0.7440,56
4,4:44,5,1gT5TGwbkkkUliNzHRIGi1,4:44,spotify:track:1gT5TGwbkkkUliNzHRIGi1,0.139000,0.261,0.852,0.000043,0.4770,-4.965,0.1580,177.997,0.4310,61
5,4:44,6,6ku4gCfjdxmYpajNONAciN,Family Feud,spotify:track:6ku4gCfjdxmYpajNONAciN,0.170000,0.543,0.721,0.000001,0.5900,-6.805,0.3170,82.992,0.4990,56
6,4:44,7,6Iuh3vNjpWhSeHM3RZgzQL,Bam,spotify:track:6Iuh3vNjpWhSeHM3RZgzQL,0.455000,0.639,0.788,0.000000,0.0580,-2.879,0.3500,155.913,0.8470,59
7,4:44,8,39ukbGu3UVWs74ba83T68c,Moonlight,spotify:track:39ukbGu3UVWs74ba83T68c,0.236000,0.758,0.633,0.000000,0.2720,-8.373,0.3910,88.170,0.5160,53
8,4:44,9,5oynsOy80DnodTslgaj3cr,Marcy Me,spotify:track:5oynsOy80DnodTslgaj3cr,0.542000,0.518,0.678,0.000000,0.1080,-6.760,0.3310,81.951,0.6040,55
9,4:44,10,3p2O5DDqlWdaR1l4wsHn9R,Legacy,spotify:track:3p2O5DDqlWdaR1l4wsHn9R,0.522000,0.425,0.858,0.000000,0.0597,-7.105,0.3960,162.171,0.6060,51


In [60]:
list_test = list(map(get_album_data, ['City Girls', 'The Beatles', 
                                      'XXXTENTACION', 'Taylor Swift',
                                     'J. Cole', 'Kendrick Lamar', 'Adele',
                                     'Bruno Mars', 'Rihanna', 'Travis Scott',
                                     'Frank Ocean', 'Ariana Grande',
                                     'Khalid']))

In [63]:
spotify_df = pd.concat(list_test)

In [64]:
spotify_df.to_csv("spotify_selected_data.csv")